## Pipeline обработки запроса

### Загрузка данных и функций (достаточно выполнить 1 раз)

In [1]:
from main import setup_environment, load_data
from retriever import retriever, RetrieverOut
from planner import planner, PlannerOut
from grounder import grounder
from executor import executor

from openai import OpenAI
from config import PipelineConfig


api_key, db_path = setup_environment()
db = load_data(db_path, wave_filter=["2025-03"])    # Фильтр по последней волне

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

2025-11-12 16:06:37 | INFO     | main                 | Environment loaded: DB_PATH=C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-12 16:06:37 | INFO     | main                 | Loading data from C:\Users\kateu\Documents\Ivanovs Index\V2\DB\25W3\db_CORRECTED.parquet
2025-11-12 16:06:38 | INFO     | main                 | Full dataset shape: (26505646, 5)
2025-11-12 16:06:38 | INFO     | main                 | Filtered by waves: ['2025-03'], new shape: (672405, 5)


### Настройка используемых моделей и их температур

Если нужно поменять модель, это делается здесь

In [ ]:
PPL_cfg = PipelineConfig.setup(
    df=db, client=client,
    # параметры ретривера
    retriever_params={
        "model": "openrouter/polaris-alpha",
        # "model": "alibaba/tongyi-deepresearch-30b-a3b:free",
        "temperature": 0.4
    },
    # параметры планировщика
    planner_params={
        "model": "meta-llama/llama-3.3-70b-instruct:free",
        # !!! "model": "deepseek/deepseek-chat-v3.1", # Платная модель
        "temperature": 0.4
    }
)

### Еще варианты моделей
# "alibaba/tongyi-deepresearch-30b-a3b:free"
# "meta-llama/llama-4-maverick:free"
# !!! "deepseek/deepseek-chat-v3.1" # Платная модель

### Retriever

Извлекает релевантные вопросы на основе запроса пользователя и всего набора вопросов

Пользовательский запрос

In [3]:
# user_query = "Я хочу посчитать индекс потребительской уверенности по потребителям из москвы"
# user_query = "Я хочу посчитать размер средних сбережений и норму сбережений среди тех, у кого они есть"
user_query = "Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"
# user_query = "Мне нужны все вопросы, связанные с автотранспортом (сроки владения, предпочтения по маркам, планы по покупке и пр.)"

Обращение к LLM

> ! Возможны проблемы парсинга ответов

In [4]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

# retriever_out = retriever(user_query, PPL_cfg)
# retriever_out.save("cur_retrieved.json")

In [5]:
retriever_out = RetrieverOut.load("cur_retrieved.json")
print(retriever_out)

1. '[C6_offline] Как часто Вы совершали покупки в этих магазинах в последнем месяце? @ Пятерочка'
	Reason: Позволяет выделить фактических текущих покупателей Пятерочки и задать порог частоты для определения «клиента».
2. '[C6_offline] Как часто Вы совершали покупки в этих магазинах в последнем месяце? @ Чижик'
	Reason: Позволяет выделить фактических текущих покупателей Чижика по аналогичному критерию.
3. '[J8_offline] Оцените уровень цен в магазинах (1 - самые низкие цены, 5 - самые высокие) @ Пятерочка'
	Reason: Помогает понять, как кросс-клиенты воспринимают ценовой уровень Пятерочки.
4. '[J8_offline] Оцените уровень цен в магазинах (1 - самые низкие цены, 5 - самые высокие) @ Чижик'
	Reason: Позволяет сравнить восприятие цен Чижика у тех же респондентов и интерпретировать причины взаимопроникновения.
5. '[Q115] В каких магазинах Вы делаете покупки?'
	Reason: базовый вопрос для расчета взаимопроникновения на уровне «пользовался/не пользовался». Пересечение ответов «Чижик» и «Пятерочк

### Planner

Строит план на основе пользовательского запроса и набора релевантных ответов, к которым подмешиваются их ответы

В план включаются команды из [`capability_spec.py`](./capability_spec.py)

In [6]:
# Сохранение, чтобы не делать 1 и тот же запрос кучу раз:
# можно 1 раз сохранить, а далее только читать

planner_out = planner(user_query, retriever_out, PPL_cfg)
planner_out.save("cur_planned.json")

2025-11-12 16:06:47 | INFO     | planner              | Starting planner for query: Я хочу посчитать долю взаимопроникновения клиентов Чижика и Пятерочки...
2025-11-12 16:06:47 | DEBUG    | planner              | Working with 7 questions
2025-11-12 16:06:47 | DEBUG    | planner              | Generated prompt of length: 6823
2025-11-12 16:06:47 | DEBUG    | planner              | Calling LLM with model: deepseek/deepseek-chat-v3.1
2025-11-12 16:06:49 | INFO     | httpx                | HTTP Request: POST https://openrouter.ai/api/v1/responses "HTTP/1.1 200 OK"
2025-11-12 16:06:54 | DEBUG    | planner              | Plan received: 7 steps
2025-11-12 16:06:54 | DEBUG    | planner              | Plan analysis: Для расчета доли взаимопроникновения клиентов Чижика и Пятерочки нужно определить: 1) общее количество клиентов каждого магазина, 2) количество клиентов, которые посещают оба магазина. Используем вопрос [Q115] 'В каких магазинах Вы делаете покупки?' как основной для определения клие

In [ ]:
planner_out = PlannerOut.load("cur_planned.json")
print(planner_out)

### Grounder

Привязка шагов плана к имеющимся функциям (из [`operations.py`](./operations.py))

In [ ]:
grounder_out = grounder(planner_out)

### Executor

Валидирует план, осуществляет топологическую сортировку, выполняет последовательность шагов

In [ ]:
ctx = {"dataset": db}
ctx = executor(grounder_out, ctx)

Далее можно извлекать созданные таблицы из `ctx`

In [ ]:
opts = [f"{i}. '{n}'" for i, n in enumerate(ctx.keys(), start=1)]
opts = "\n".join(opts)

print(f"Доступные варианты:\n{opts}")

Пока что извлекать и смотреть можно только ручками...

In [ ]:
ctx["pivot_n47"]